# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [8]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
#import time
import requests
import schedule
import re
import json
import time
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [22]:
def get_book_data(book_url: str) -> dict:
    """
    Функция, которая возвращает словарь с данными о полученной книге
    Args: book_url (str) путь к странице с книгой
    Returns: Словарь {
        title: Название книги
        price: Стоимость книги
        raiting: Рейтинг книги
        count_available: Количество в наличии
        description: Описание
        upc: 
        product_type
        tax
        numbers_of_review
    }    
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    res_dict = {}
    response = requests.get(book_url)
    soup = BeautifulSoup(response.text, "html.parser") 
    tag_div = soup.find("div", {"class":"col-sm-6 product_main"})
    reg_price = re.compile(r"\d+(?:\.\d+)?")
    reg_available = re.compile(r"\((\d+).*?\)")
    res_dict["title"] = soup.find("title").get_text().split('|')[0].strip()
    res_dict["price"] = re.findall(reg_price, tag_div.find("p", {"class":"price_color"}).text)[0]
    classes = tag_div.find_all("p")
    for next_class in classes:
        if next_class.get("class")[0] == "star-rating":
            res_dict["rating"] =next_class.get("class")[1]
            break    
    instock_availability = tag_div.find("p", {"class":"instock availability"}).text.strip()
    res_dict["count_available"] = re.findall(reg_available, instock_availability)[0]
    res_dict["description"] = soup.find("meta", {"name": "description"}).get("content").strip()
    tables_rows = soup.find("table", {"class":"table table-striped"}).find_all("tr")
    for row in tables_rows:
        if row.find("th").text == "UPC":
            res_dict["upc"] = row.find("td").text
        elif row.find("th").text == "Product Type":
            res_dict["product_type"] = row.find("td").text
        elif row.find("th").text == "Tax":
            res_dict["tax"] = re.findall(reg_price, row.find("td").text)[0] 
        elif row.find("th").text == "Number of reviews":
            res_dict["number_of_reviews"] = row.find("td").text

    return res_dict
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [23]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '51.77',
 'rating': 'Three',
 'count_available': '22',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And w

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [24]:
def scrape_books(is_save=True):
    """
    Функция для получения данных по всем книгам и записи их 
    в файл artifacts/books_data.txt 
    Args: 
    is_save по-умолчанию True, определяет будет ли записаны данные 
    в файл books_data.txt
    Returns:
    Возвращает список из словарей с данными по каждой книге
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    list_book = []
    base_url ="http://books.toscrape.com/catalogue/"
    next_href = "page-1.html"
    response = requests.get(base_url + next_href)
    soup = BeautifulSoup(response.text, "html.parser")
    reg_pages = re.compile(r"of\s(\d+)")
    pager = re.findall(reg_pages, soup.find("li", {"class":"current"}).text)[0] 
    count_page = 1
    while count_page < int(pager):
        all_books_on_page = soup.find("ol", {"class":"row"}).find_all("li")
        for next_book in all_books_on_page:
            book_href = next_book.find("a").get("href")
            data_book = get_book_data(base_url + book_href)
            list_book.append(data_book)
        next_href = (
            soup.find("ul", {"class":"pager"})
            .find("li", {"class":"next"})
            .find("a").get("href")
            )
        response = requests.get(base_url + next_href)
        soup = BeautifulSoup(response.text, "html.parser")
        count_page += 1

    if is_save:
        with open("./artifacts/books_data.txt", "w", encoding="utf-8") as f:
            json.dump(list_book, f, ensure_ascii=False, indent=2)

    return list_book
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.every().day().at("19:00").do(scrape_books)

while True:
    schedule.run_pending()  # Проверяет, не пора ли выполнить задачу
    time.sleep(1) 
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

In [25]:
def test_get_book_data():
    book_url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    dict_data = get_book_data(book_url)    
    assert isinstance(dict_data, dict), "Тип не словарь"
    assert len(dict_data.keys()) == 9, "Не совпадает количество ключей в словаре" 
    assert dict_data["title"] == "A Light in the Attic"
    assert dict_data["price"] == "51.77"
    assert dict_data["rating"] == "Three"
    assert dict_data["count_available"] == "22"    
    assert dict_data["upc"] == "a897fe39b1053632"
    assert dict_data["product_type"] == "Books"
    assert dict_data["tax"] == "0.00"
    assert dict_data["number_of_reviews"] == "0" 
    print("test_get_book_data is success")     


def test_scrape_books():
    list_data = scrape_books(False)
    assert isinstance(list_data, list), "Тип не список"
    assert len(list_data) == 980, "Не все книги найдены"
    assert isinstance(list_data[0], dict), "Элементы списка не словари"
    assert len(list_data[0].keys()) == 9, "Не совпадает количество ключей" 
    expected_keys = {
        "title", 
        "price", 
        "rating", 
        "count_available", 
        "description", 
        "upc", 
        "product_type", 
        "tax", 
        "number_of_reviews"
        }
    real_keys = set(list_data[0].keys())
    assert expected_keys.intersection(real_keys) == expected_keys
    print("test_scrape_books is success")

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```